<a href="https://colab.research.google.com/github/abdulsamadkhan/Tutorial/blob/main/text_generation_using_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This tutorial demonstrates how to generate text using a character-based RNN.
# TASK:
 Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e").




## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

### Download the Shakespeare dataset



In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


### Read the data



In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# Obtain the set of unique characters in the given text file
vocab = sorted(set(text))

# Print the number of unique characters in the vocabulary
print(f'There are {len(vocab)} unique characters in the given text.')
# Print the unique characters in the vocabulary
print(vocab)

There are 65 unique characters in the given text.
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
ids_from_chars.get_vocabulary()

['[UNK]',
 '\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

It converts from tokens to character IDs:

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [14]:
text[:10]

'First Citi'

In [15]:
import tensorflow as tf

# Suppose you have some data in the form of NumPy arrays
features = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
labels = np.array([0, 1, 0, 1, 0])

# Create a dataset using from_tensor_slices
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

# Print the elements of the dataset
# Accessing individual components of the dataset
for element in dataset:
    feature_value, label_value = element
    print("Feature:", feature_value.numpy(), "Label:", label_value.numpy())


Feature: 1.0 Label: 0
Feature: 2.0 Label: 1
Feature: 3.0 Label: 0
Feature: 4.0 Label: 1
Feature: 5.0 Label: 0


In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([19, 48, 57, 58, 59])>

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for element in ids_dataset:
  print(element)
  break

tf.Tensor(19, shape=(), dtype=int64)


In [18]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [19]:
#seq_length = 100
seq_length = 10

In [20]:
text[:20]

'First Citizen:\nBefor'

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [21]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(2):
  print(chars_from_ids(seq))

tf.Tensor([b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z'], shape=(11,), dtype=string)
tf.Tensor([b'e' b'n' b':' b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' '], shape=(11,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [22]:
for seq in sequences.take(5):
  print(chars_from_ids(seq).numpy())

[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z']
[b'e' b'n' b':' b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ']
[b'w' b'e' b' ' b'p' b'r' b'o' b'c' b'e' b'e' b'd' b' ']
[b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h' b'e' b'r']
[b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's']


For training you'll need a dataset of `(input, label)` pairs. Where `input` and
`label` are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [23]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [24]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [25]:
dataset = sequences.map(split_input_target)

In [26]:
for input_example, target_example in dataset.take(1):
    print("Input :", chars_from_ids(input_example).numpy())
    print("Target:", chars_from_ids(target_example).numpy())

Input : [b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i']
Target: [b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z']


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [27]:
# Batch size
#BATCH_SIZE = 64

BATCH_SIZE = 4

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(4, 10), dtype=tf.int64, name=None), TensorSpec(shape=(4, 10), dtype=tf.int64, name=None))>

In [28]:
iterator=iter(dataset)
for batch in iterator:
  features_batch, labels_batch = batch
  print("Features:", features_batch.numpy())
  print("Labels:", labels_batch.numpy())
  print("---------------------")
  break

Features: [[11  2 59 47 44  2 41 57 44 40]
 [ 2 59 47 44  2 46 54 54 43  2]
 [47 44  2 41 54 43 64  7  2 48]
 [ 2 64 54 60  2 58 47 54 60 51]]
Labels: [[ 2 59 47 44  2 41 57 44 40 58]
 [59 47 44  2 46 54 54 43  2 46]
 [44  2 41 54 43 64  7  2 48 43]
 [64 54 60  2 58 47 54 60 51 43]]
---------------------


## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [29]:
## Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256
#embedding_dim = 3

# Number of RNN units
rnn_units = 1024

In [30]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    #print('input',x)
    #print("-----------")
    x = self.embedding(x, training=training)
    #print('embedd',x)
    #print("-----------")

    if states is None:
      states = self.gru.get_initial_state(x)
      #print('gru1',states)
      #print("-----------")

    x, states = self.gru(x, initial_state=states, training=training)
    #print('gru2  states',states)
    #print('gru_2_x',x)
    #print("-----------")

    x = self.dense(x, training=training)
    #print('dense',x)


    if return_state:
      return x, states
    else:
      return x

In [31]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(4, 10, 66) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [41]:
model.summary()
#example_batch_predictions[0]

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [36]:
sampled_indices

array([10, 52, 52, 38, 47, 19, 53, 43, 40, 24])

Decode these to see the text predicted by this untrained model:

In [39]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b" obey'd. P"

Next Char Predictions:
 b'3mmYhFndaK'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [40]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [42]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (4, 10, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189433, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [43]:
tf.exp(example_batch_mean_loss).numpy()

65.985374

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [44]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [45]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [48]:
EPOCHS = 2

In [49]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/2
25349/25349 [==============================] - 156s 6ms/step - loss: 1.5738
Epoch 2/2
25349/25349 [==============================] - 154s 6ms/step - loss: 1.5826


## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


In [58]:
sikp_ids = (ids_from_chars(['[UNK]'])[:,None])
values=[-float('inf')]*len(sikp_ids)
values

[-inf]

The following makes a single step prediction:

In [106]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    print(inputs)
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')

    input_ids = self.ids_from_chars(input_chars).to_tensor()

    print("Input IDs:", input_ids.numpy())  # Assuming eager execution is enabled


    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [107]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [108]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO'])
result = [next_char]

for n in range(10):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor([b'ROMEO'], shape=(1,), dtype=string)
Input IDs: [[31 28 26 18 28]]
tf.Tensor([b':'], shape=(1,), dtype=string)
Input IDs: [[11]]
tf.Tensor([b'\n'], shape=(1,), dtype=string)
Input IDs: [[1]]
tf.Tensor([b'N'], shape=(1,), dtype=string)
Input IDs: [[27]]
tf.Tensor([b'a'], shape=(1,), dtype=string)
Input IDs: [[40]]
tf.Tensor([b'y'], shape=(1,), dtype=string)
Input IDs: [[64]]
tf.Tensor([b' '], shape=(1,), dtype=string)
Input IDs: [[2]]
tf.Tensor([b'o'], shape=(1,), dtype=string)
Input IDs: [[54]]
tf.Tensor([b'f'], shape=(1,), dtype=string)
Input IDs: [[45]]
tf.Tensor([b' '], shape=(1,), dtype=string)
Input IDs: [[2]]
ROMEO:
Nay of s 

________________________________________________________________________________

Run time: 0.32229089736938477
